In [1]:
!pip install --upgrade openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/3e/d3/309769dad11d5f75b81c7252d9dc849ed440d0921215e759af169054f3b6/openai-1.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/a2/65/6940eeb21dcb2953778a6895281c179efd9100463ff08cb6232bb6480da7/httpx-0.25.2-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/221.4 kB ? eta -:--:--
   ---------------------------------------- 221.4/221.4 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.0 kB ? eta -:--:--
   --------------

In [2]:
import os
import openai
import glob
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [21]:
def debug_code_with_gpt(description, code_content, API_KEY, few_shot, CoT):
    if few_shot == '0' and CoT == 'n':
        # for the prompt, use prompts/zeroShot_noCoT.txt as the prompt
        with open('prompts/zeroShot_noCoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    elif few_shot == '0' and CoT == 'c':
        with open('prompts/zeroShot_CoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    elif few_shot == 'f' and CoT == 'n': 
        with open('prompts/fewShot_noCoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    elif few_shot == 'f' and CoT == 'c':
        with open('prompts/fewShot_CoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    else:
        raise ValueError("few_shot should be '0' or 'f', CoT should be 'n' or 'c'.")

    try:
        client = openai.OpenAI(api_key=API_KEY)
        pass
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": f"{prompt}"}
            ],
            max_tokens=2048  # Adjust as needed
        )
        output = response.choices[0].message.content
#         lines = output.split('\n')
#         # Remove the first and last lines
#         modified_content = '\n'.join(lines[1:-1])
#         return modified_content
        return output
    except Exception as e:
        print("Error during API call:", e)
        return None

def process_file(file_path, target_directory, api_key, few_shot, CoT):
    description_path = 'data/description/' + ''.join(file_path.split('/')[-1].split('_')[:-1]).split('.')[0] + '.txt'
    # if the description file is not found, skip this file
    try:
        with open(description_path, 'r') as file:
            description = file.read()
    except:
        print("Description file not found:", description_path)
        return None
    
    with open(file_path, 'r') as file:
        code_content = file.read()

    debugged_content = debug_code_with_gpt(description, code_content, api_key, few_shot, CoT)

    if debugged_content:
        target_file_path = os.path.join(target_directory, os.path.basename(file_path))
        with open(target_file_path, 'w') as file:
            file.write(debugged_content)
#         print(f"Debugged code written to {target_file_path}")

def process_files_in_parallel(source_directory, target_directory, api_key, few_shot, CoT, max_workers=500):
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    file_paths = glob.glob(os.path.join(source_directory, '*.java'))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_file, file_path, target_directory, api_key, few_shot, CoT)\
                   for file_path in file_paths]
        for future in tqdm(as_completed(futures), total=len(file_paths)):
            future.result()  # This will re-raise any exception caught during process_file execution

In [22]:
def main(few_shot, CoT):
    with open('API_KEY.txt', 'r') as file:
        api_key = file.read().strip()
    source_dir = 'data/formatted/buggy_codes'  # Source directory
#     source_dir = 'data/test/bug_codes_tests' # Source dir for pilot test
    target_dir = f'data/formatted/debugged_codes_{few_shot}_{CoT}'  # Target directory
#     target_dir = f'data/test/debugged_GPTBuggyCodes_tests_{few_shot}_{CoT}' # Target dir for pilot test

    process_files_in_parallel(source_dir, target_dir, api_key, few_shot, CoT)

In [23]:
if __name__ == "__main__":
    # few_shot: '0' (zero-shot) or 'f' (few-shot)
    # CoT: 'n' (no Chain-of-Thought) or 'c' (with Chain-of-Thought)
    
    # RUN ALL COMBINATIONS
    few_shot = '0'
    CoT = 'n'
    main(few_shot, CoT)
    
#     few_shot = '0'
#     CoT = 'c'
#     main(few_shot, CoT)
    
#     few_shot = 'f'
#     CoT = 'n'
#     main(few_shot, CoT)
    
#     few_shot = 'f'
#     CoT = 'c'
#     main(few_shot, CoT)

Description file not found: data/description/buggycodes\01 Matrix.txt
Description file not found: data/description/buggycodes\1-bit and 2-bit Characters.txt
Description file not found: data/description/buggycodes\132 Pattern.txt
Description file not found: data/description/buggycodes\2 Keys Keyboard.txt
Description file not found: data/description/buggycodes\24 Game.txt
Description file not found: data/description/buggycodes\3Sum With Multiplicity.txt
Description file not found: data/description/buggycodes\4Sum.txt
Description file not found: data/description/buggycodes\A Number After a Double Reversal.txt
Description file not found: data/description/buggycodes\Accounts Merge.txt
Description file not found: data/description/buggycodes\Add Binary.txt
Description file not found: data/description/buggycodes\Add One Row to Tree.txt
Description file not found: data/description/buggycodes\Add Minimum Number of Rungs.txt
Description file not found: data/description/buggycodes\Add Digits.txt
D

100%|██████████████████████████████████████████████████████████████████████████| 1766/1766 [00:00<00:00, 252329.79it/s]
